## Исследование надёжности заёмщиков
Цель проекта:
* На основе данных кредитного банка, нужно выяснить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Задачи проекта:
* Изучить общую информацию
* Провести предобработку данных, найти дубликаты, пропуски и обработать их
* Заменить тип данных на соответствующие хранящимся данным
* Провести лемматизацию некоторых столбцов
* Провести категоризацию данных и ответить на поставленные вопросы:
    * Есть ли зависимость между наличием детей и возвратом кредита в срок?
    * Есть ли зависимость между семейным положением и возвратом кредита в срок?
    * Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    * Как разные цели кредита влияют на его возврат в срок?

1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропущенных значений](#null)
    * [Изменение типов данных](#types)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemma)
    *[Категоризация данных](#cohorts)
3. [Ответы на вопросы](#answers)
    * [Зависимость между наличием детей и возвратом кредита в срок](#childratioreturn)
    * [Зависимость между семейным положением и возвратом кредита в срок](#maritalstatus)
    * [Зависимость между уровнем дохода и возвратом кредита в срок](#incomeratio)
    * [Как разные цели кредита влияют на его возврат в срок](#goalofloan)
4. [Вывод](#conclusion)

<a id='start'></a>
### Шаг 1. Изучение общей информации. 

In [1]:
import pandas as pd
credit_data = pd.read_csv('/Users/valeriyaniskorodova/Y.Praktikum/Sprint 1/data (1).csv')
credit_data.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
credit_data.columns # Получение информации о названиях столбцов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Сразу можно заметить:
 - отрицательное количество дней трудового стажа;
 - в стобце образование разный регистр нескольких букв у данных;
 - в названиях столбцов нет лишних пробелов, столбцы названы корректно; 

In [3]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов и 21525 строк


Количество значений в столбцах с общим трудовым стажем и ежемесячным доходом различаются, что говорит о присутствии строк с пропусками

Замечу, что данные в столбцах с общим трудовым стажем и ежемесячным доходом имеют тип float 64. Для days_employed не корректно использовать этот тип данных, так как общий трудовой стаж представлен в днях, а значит не может быть представлен в десятичном виде. Предполагаю, что данная ошибка возникла из-за того, что данные были представлены не в днях, а в часах, например, и поэтому при переводе из часов в дни возникла десятичная дробь.

На счет столбца total_income нельзя утверждать однозначно, так как доход мы можем записывать в виде рублей в целой части, а копеек в десятичной части. При этом проводит математические операции удобнее с целыми числами, нежели чем с числами с плавающей запятой. 

In [4]:
credit_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видно так же, что в столбце children есть отрицательное значение, а в dob_years нулевые значения

### Вывод

Из промежуточных выводов можно выделить несколько проблем, которые нужно решать:
- Пропуски;
- Отрицательные данные;
- Данные с разным регистром нескольких букв;
- Некоторые данные имеют тип float 64

Для ответа на поставленные вопросы нам будут важны столбцы children, family_status, total_income и purpose

<a id='preprocessing'></a>
### Шаг 2. Предобработка данных

<a id='null'></a>
### Обработка пропусков

In [5]:
credit_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Еще раз убедились, где находятся пропуски, столбцы total_income и days_employed

Проверим, есть ли зависимость между столбцами

In [61]:
credit_data[credit_data['days_employed'].isnull() &\
            credit_data['total_income'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Отсюда вижу, что пропуски в этих двух столбцах совпадают

Для того, чтобы понять, как правильно обработать пропуск, надо определить механизм формирования их.

Люди, которые занимают руководящие должности, а так же имеют образование высшее образование чаще, чем другие не отвечают на вопрос о своих доходах. Поскольку доход имеет зависимость с образованием и должностью, то пропуски нельзя считать совершенно случайными. Так как у нас есть информация в таблице об образовании и занимаемой должности, то может выполнятся гипотеза MAR. При таком механизме формирования пропусков, исключение их будет вполне приемлемо. Но удаление пустых строк приведет к потере 10 % данных из таблицы, что повлияет на вывод исследования. Заполнение нулями тоже приведет к искажению результата, поэтому я решила заполнить пропуски средним значением

Но, для начала, избавлюсь от отрицательных значений в столбце days_employed

In [7]:
credit_data[credit_data['days_employed'] < 0 ]['days_employed'].count()

15906

Отрицательных значений в таблице большинство. Откуда они могли появиться?

Обычно запись в трудовой производится так: ДД.ММ.ГГГГ

В нашей таблицы данные хранятся в днях, а значит был произведен расчет, а отрицательные значения появились, потому что  могли вычитать из даты начала работы дату последнего дня работы. Даже при использовании формулы в системе, сотрудники могли не придать особо внимание тому, куда записывать дату принятия на работу и дату уволнения.

Избавимся от отрицательных значений:

In [8]:
credit_data['days_employed'] = abs(credit_data['days_employed'])
credit_data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В столбце days_employed присутствуют аномальные отклонения, а именно большое колличество дней работы. Если человек работает в среднем от 20 до 65 лет. В РФ в среднем 247 рабочих дней в году, а значит в среднем чеслове не может работать больше 11115 дней за свою жизнь. Проверим, есть ли строки, которые содержат числа больше в этом столбце:

In [9]:
credit_data[credit_data['days_employed'] > 11115]['days_employed'].count()

3610

Таких значений в таблице 3610. Скорее всего так произошло, потому что эти данные брали из других источников, где подсчет ввелся не в днях, а часах или минутах. Посмотрим, какое самое большое значение у нас существует:

In [10]:
credit_data['days_employed'].max()

401755.40047533

Так как таких данных в нашей таблице всего 3610, а это около 17%, то исключать их - значит потерять большую часть данных. Переводить эти данные из часов или минут в дни я не захотела, так как нет уверенности в том, какая система измерения была. Поэтому приняла решение заменить их на среднее значение.

Теперь заполним пропуски и аномальные данные средним значением

In [63]:
days_avg = credit_data[credit_data['days_employed'] <= 11115]\
['days_employed'].mean()
credit_data['days_employed'] = credit_data['days_employed'].\
fillna(days_avg)


In [12]:
#Функция возвращает среднее значение столбца, если ему встретилось число больше 11115
def remove_anomal(row):
    
    days = row['days_employed']
    if days > 11115:
        return days_avg
    else:
        return days

In [13]:
credit_data['days_employed'] = credit_data.apply(remove_anomal, axis=1)
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2244.966411,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверим теперь максимальное :

In [14]:
credit_data['days_employed'].max()

11109.782505618114

Теперь максимальное число не привышает 11115


Теперь заполним пропуски



Найдем среднее значение для каждого типа занятости в нашей таблице и заполним пропуски

In [15]:
credit_data.groupby("income_type")['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [64]:
credit_data['total_income'] = credit_data['total_income'].\
fillna(credit_data.groupby('income_type')['total_income'].\
       transform('mean'))

In [17]:
credit_data[credit_data["income_type"] == 'сотрудник' ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
...,...,...,...,...,...,...,...,...,...,...,...,...
21515,1,467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21519,1,2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


В этом столбце решила взять медиану, так как среднее значение смещено в большую сторону

Убедимся, что пропущенных значений больше нет

In [18]:
credit_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [19]:
age_avr = credit_data[credit_data['dob_years'] != 0]['dob_years'].mean()


In [20]:
def replace_age(row):
    age = row['dob_years']
    
    if age == 0:
        return age_avr
    return age

In [21]:
credit_data['dob_years'] = credit_data.apply(replace_age, axis=1)

Избавимся от отрицательного значения в столбце children. Выдвигаю гипотезу, что это значение возникло по ошибки ввода во время заполнения анкеты в банке. Проверим, сколько таких результатов в нашей таблице:

In [22]:
credit_data[credit_data['children'] < 0]['children'].count()

47

47 строк в столбце children имеют отрицательное значение. Эти данные дают нам меньше 1% от всех данных в этом столбце. Но я решила их не удалять, а заменить на положительные значения в связи с гипотезой.

In [23]:
credit_data['children'] = abs(credit_data['children'])

Проверим еще раз нет ли аномальных значений в нашей таблице после проведенных операций

In [24]:
credit_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,2244.966411,43.497479,0.817236,0.972544,0.080883,1.673959e+05
std,1.379876,1756.470646,12.218166,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2244.966411,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,2447.430087,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,20.000000,11109.782506,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [25]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


### Вывод

В этом разделе избавились от артифактов, аномальных значений и пропусков. Заметим, что после замены нулевых значений в столбце dob_years, у нас поменялся тип данных на float64

<a id='types'></a>
### Замена типа данных

Столбец days_employed не может иметь тип float 64, потому что данные представлены в днях, а значит не могут иметь десятичную часть. Столбец total_income тоже имеет тип float 64, его можно оставить без измененений, но, тип int смотрибельнее. Более того, при замене типа данных для total_income, десятичная часть не повлияет на конечный результат, что так же позволяет нам избавится от нее. Так же вернем столбцу dob_years тип int64

Так как нам нужно перевести из float в int, то я не использовала метод to_numeric(), так как он переводит из строкового типа в численный,так еще и при переводе все числа будут иметь тип float. Нам это не нужно. Так как нам нужно перевести в определенное значение, я использовала метод astype()

In [26]:
credit_data['days_employed'] = credit_data['days_employed'].astype('int')
credit_data['total_income'] = credit_data['total_income'].astype('int')
credit_data['dob_years'] = credit_data['dob_years'].astype('int')
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [27]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Теперь с данными удобно работать и выглядят они хорошо)

<a id='duplicates'></a>
### Обработка дубликатов


Приведем значения в столбце education к одному регистру

In [28]:
credit_data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [29]:
credit_data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [30]:
credit_data['education'] = credit_data['education'].str.lower()
credit_data['family_status'] = credit_data['family_status'].str.lower()



    Проверим:

In [31]:
credit_data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [32]:
credit_data['family_status'].unique()


array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)


    Теперь посмотрим на наличие дубликатов в нашей таблице

In [33]:
credit_data.duplicated().sum()

71


    
Видим, что у нас присутствует 71 дубликата в таблице. Найдем их

In [34]:
credit_data[credit_data.duplicated() == True].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2244,41,среднее,1,женат / замужем,0,F,сотрудник,0,161380,покупка жилья для семьи
3290,0,2244,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
4182,1,2244,34,высшее,0,гражданский брак,1,F,сотрудник,0,161380,свадьба
4851,0,2244,60,среднее,1,гражданский брак,1,F,пенсионер,0,137127,свадьба
5557,0,2244,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
6312,0,2244,30,среднее,1,женат / замужем,0,M,сотрудник,0,161380,строительство жилой недвижимости
7808,0,2244,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7921,0,2244,64,высшее,0,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7938,0,2244,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
8583,0,2244,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,137127,дополнительное образование



Видим, что дубликаты у нас в столбце days_emloyed и total_income. Я думаю, они возникли из-за того, что между этими двумя столбцами есть зависимость в виде формулы. Но, так как, 71 строчки это меньше 1% от наших данных, то удаление дубликатов не приведет к потере важных данных и большому влиянию на исследование данных.

In [35]:
new_credit_data = credit_data.drop_duplicates().reset_index(drop=True)

Проверим, остались ли у нас дубликаты в таблице:

In [36]:
new_credit_data.duplicated().sum()

0

### Вывод

Избавилась от дубликатов.

<a id='lemma'></a>
### Лемматизация

Посмотрим на уникальные значения в столбцах

Проверим, что происходит в других столбцах

In [37]:
new_credit_data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [38]:
new_credit_data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Далее, для того, чтобы нам проще было категоризировать данные, нужно провести лемматизацию столбца purpose, так как причина займа во многих строках одинакова, но записана по-разному

In [39]:
from pymystem3 import Mystem
m = Mystem()

In [65]:
new_credit_data['lemm_purpose'] = new_credit_data['purpose'].\
apply(m.lemmatize)
new_credit_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm_purpose,income_group,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",средний,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",высокий,образование
4,0,2244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",средний,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",высокий,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",средний,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",средний,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",средний,недвижимость


### Вывод

Я провела лемматизацию данных, и теперь будет намного проще категоризировать их

<a id='cohorts'></a>
### Категоризация данных

Посмотрим зависимость количества детей и вовзращения кредита. 

In [41]:
new_credit_data.groupby(by='children')['debt'].value_counts()

children  debt
0         0       13028
          1        1063
1         0        4410
          1         445
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
20        0          68
          1           8
Name: debt, dtype: int64

In [66]:
children_debt_ratio = new_credit_data.groupby(by='children')['debt'].\
mean()
round(children_debt_ratio*100, 2)

children
0      7.54
1      9.17
2      9.45
3      8.18
4      9.76
5      0.00
20    10.53
Name: debt, dtype: float64

Как видим, чем больше детей имеет человек, тем больше вероятность невозврата кредита в срок. 

Рассмотрим теперь зависимость семейного положения и возврата кредита в срок.

In [67]:
family_debt = new_credit_data.groupby(by='family_status')['debt'].\
value_counts()
family_debt

family_status          debt
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3763
                       1         388
женат / замужем        0       11408
                       1         931
не женат / не замужем  0        2536
                       1         274
Name: debt, dtype: int64

In [68]:
family_debt_ratio = new_credit_data.groupby(by='family_status')['debt'].\
mean()
round(family_debt_ratio*100, 2)

family_status
в разводе                7.11
вдовец / вдова           6.57
гражданский брак         9.35
женат / замужем          7.55
не женат / не замужем    9.75
Name: debt, dtype: float64

Отсюда видим, что больше максимальный процент людей, не вернувших кредит в срок, не женаты/не замужем или состоят в гражданском браке. Самый малый процент у людей, которые потеряли супруга/супругу.

Можем сделать вывод, что одинокие люди, не состоящие в браке ни разу, имеют склонность не возвращать кредит в срок, то же самое касается людей, состоящие в гражданском браке. Скорее всего так происходит из-за того, что у человек нет финансовой поддержки от партнера, а в гражданском браке у каждого свой собственный доход и может не быть общего счета. С другой стороны одинокие люди, которые остались без партнера из-за развода или они стали вдовой/вдовец, имеют наименьший процент по не возврату кредита в срок.

Доход у людей очень разный, для удобство разделим его на 3 группы: низкий, средний, высокий. Средний доход в России зависит от региона проживания. Так как в задании не сказано, какой банк и где он расположен, возьмем за средний доход зарплату от 90.000 до 180.000.

In [45]:
def income_group(row):
    
    income = row['total_income']
    
    if income < 80000:
        return 'низкий'
    if 80000 < income < 160000:
        return 'средний'
    if income > 160000:
        return 'высокий'

In [69]:
new_credit_data['income_group'] = new_credit_data.\
apply(income_group, axis=1)

In [70]:
income_debt = new_credit_data.groupby('income_group')['debt'].\
value_counts()
income_debt

income_group  debt
высокий       0       9162
              1        773
низкий        0       2102
              1        174
средний       0       8449
              1        794
Name: debt, dtype: int64

In [72]:
income_debt_ratio = new_credit_data.groupby('income_group')['debt'].\
mean()
round(income_debt_ratio * 100, 2)

income_group
высокий    7.78
низкий     7.64
средний    8.59
Name: debt, dtype: float64

Видим, что самый высокий процент людей, не вовзращающих кредит в срок находятся в группе среднего дохода. Процент еще зависит от того, как я распределю людей по группам, то есть какой доход я выберу средним. Если варьировать эти цифры, то все равно средний доход остается на лидирующих позициях. Мы так же можем заметить, что впринципе люди, со средним доходом активнее берут кредит. То есть вовзрат кредита в срок очень мало зависит от дохода. 

Рассмотрим зависимость целью и вовзрата кредита вовремя. Для удобства создам новый столбец, в котором обобщим цель кредита.

In [49]:
def purpose_group(row):
    purpose = row['lemm_purpose']
    #Функция возвращает общее название группы, если такое слово встречается в столбце с lemm_purpose
    if 'жилье' in purpose or 'недвижимость' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if  'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    

In [73]:
new_credit_data['purpose_group'] = new_credit_data.\
apply(purpose_group,axis=1)
new_credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm_purpose,income_group,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",средний,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",средний,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",высокий,образование
4,0,2244,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",средний,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",высокий,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",высокий,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",средний,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",средний,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",средний,недвижимость


In [74]:
purpose_debt = new_credit_data.groupby('purpose_group')['debt'].\
value_counts()
purpose_debt

purpose_group  debt
автомобиль     0        3903
               1         403
недвижимость   0       10029
               1         782
образование    0        3643
               1         370
свадьба        0        2138
               1         186
Name: debt, dtype: int64

In [75]:
purpose_debt_ratio = new_credit_data.groupby('purpose_group')['debt'].\
mean()
round(purpose_debt_ratio*100, 2)

purpose_group
автомобиль      9.36
недвижимость    7.23
образование     9.22
свадьба         8.00
Name: debt, dtype: float64

Люди, которые берут кредит на образование и на автомобиль имеют больший процент невозврата кредита в срок.

### Вывод

Провела категоризацию данных, получила таблички с процентом заемщиков, не вернувших кредит в срок

<a id='answers'></a>
### Шаг 3. Ответы на вопросы

<a id='childratioreturn'></a>
- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [85]:
overall_child = new_credit_data.groupby('children').\
agg({'children': 'count', 'debt': 'mean'})
overall_child = overall_child.\
rename(columns={'children': 'Всего заемщиков',
                'debt': '% должников'})
overall_child.style.format({'% должников': '{:.2%}'})

,Всего заемщиков,% должников
children,,
0,14091,7.54%
1,4855,9.17%
2,2052,9.45%
3,330,8.18%
4,41,9.76%
5,9,0.00%
20,76,10.53%


### Вывод

Да, зависимость есть, и , чем больше детей заемщик имеет, тем больше вероятность невозврата кредита в срок

<a id='maritalstatus'></a>
- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [84]:
overall_family = new_credit_data.groupby('family_status').\
agg({'family_status': 'count', 'debt':'mean'})

overall_family = overall_family.\
rename(columns={'family_status':'Всего заемщиков',
                'debt':'% должников'})
overall_family.style.format({'% должников': '{:.2%}'})



,Всего заемщиков,% должников
family_status,,
в разводе,1195,7.11%
вдовец / вдова,959,6.57%
гражданский брак,4151,9.35%
женат / замужем,12339,7.55%
не женат / не замужем,2810,9.75%


### Вывод

Из полученных цифр можно выдвинуть гипотезу, что одинокие люди или люди, находящиеся в гражданском браке, чаще не вовзращают кредит в срок из-за того, что либо не имеют финансовой подстраховки в виде партнера, либо имеют раздельный счет, несмотря на проживание вместе. С другой стороны, заемщики, которые потеряли супруга/супругу или находятся в разводе, имеют меньший процент должников.

<a id='incomeratio'></a>
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [87]:
income_overall = new_credit_data.groupby('income_group').\
agg({'income_group':'count','debt':'mean'})
income_overall = income_overall.\
rename(columns={'income_group':'Всего заемщиков',
                'debt':'% должников'})
income_overall.style.format({'% должников': '{:.2%}'})

,Всего заемщиков,% должников
income_group,,
высокий,9935,7.78%
низкий,2276,7.64%
средний,9243,8.59%


### Вывод

Из этой таблицы можно сделать вывод, что зависимость между уровнем дохода и вовзратом кредита в срок очень маленькая. Так как больше всего должников у нас наблюдается среди людей, имеющих средний доход( от 80000 до 160000), хотя, по идее, чем выше доход, тем меньше проблем вернуть кредит в срок. Значит есть другие причины, по которым у людей со средним доходом не получилось вернуть кредит в срок.

<a id ='goalofloan'></a>
- Как разные цели кредита влияют на его возврат в срок?

In [89]:
purpose_overall = new_credit_data.groupby('purpose_group').\
agg({'purpose_group':'count','debt':'mean'})

purpose_overall = purpose_overall.\
rename(columns={'purpose_group':'Всего заемщиков',
                'debt':'% должников'})
purpose_overall.style.format({'% должников': '{:.2%}'})

,Всего заемщиков,% должников
purpose_group,,
автомобиль,4306,9.36%
недвижимость,10811,7.23%
образование,4013,9.22%
свадьба,2324,8.00%


In [90]:
data_pivot_children = new_credit_data.groupby('children').\
agg({'children':'count','debt':'mean'})
data_pivot_children = data_pivot_children.\
rename(columns={'children':'Всего заемщиков','debt':'% должников'})
data_pivot_children.style.format({'% должников': '{:.2%}'})

,Всего заемщиков,% должников
children,,
0,14091,7.54%
1,4855,9.17%
2,2052,9.45%
3,330,8.18%
4,41,9.76%
5,9,0.00%
20,76,10.53%


In [104]:
data_pivot_family = new_credit_data.\
groupby('family_status').agg({'family_status':'count',
                            'debt':'mean'})
data_pivot_family = data_pivot_family.rename(columns={'family_status':'Всего заемщиков',
                                  'debt':'% должников'})
data_pivot_family.style.format({'% должников': '{:.2%}'})


,Всего заемщиков,% должников
family_status,,
в разводе,1195,7.11%
вдовец / вдова,959,6.57%
гражданский брак,4151,9.35%
женат / замужем,12339,7.55%
не женат / не замужем,2810,9.75%


In [106]:
data_pivot_income = new_credit_data.\
pivot_table(index='income_group', 
            values = 'debt', 
            aggfunc = ['count', 'mean'])
data_pivot_income = data_pivot_income.\
rename(columns={'count':'Всего заемщиков', 
                'mean':'% должников','debt':''})
data_pivot_family.style.format({'% должников': '{:.2%}'})

,Всего заемщиков,% должников
family_status,,
в разводе,1195,7.11%
вдовец / вдова,959,6.57%
гражданский брак,4151,9.35%
женат / замужем,12339,7.55%
не женат / не замужем,2810,9.75%


In [117]:
data_pivot_purpose = new_credit_data.\
groupby('purpose_group').agg({'purpose_group':'count',
                             'debt':'mean'})
data_pivot_purpose = data_pivot_purpose.\
rename(columns={'debt':'% должников'})
data_pivot_purpose.style.format({'% должников': '{:.2%}'})

,purpose_group,% должников
purpose_group,,
автомобиль,4306,9.36%
недвижимость,10811,7.23%
образование,4013,9.22%
свадьба,2324,8.00%


### Вывод

Как выяснилось, заемщики, которые брали кредит на образование и автомобиль, в большинстве своем не возвращали кредит в срок. С образованием выдвинуть гипотезу довольно не сложно. Если студенту есть уже 18 лет, он может оформить кредит на себя, но не всегда студенты могут найти работу сразу после окончания университета, отсюда и возникает задолжность. В России есть специальные программы, по которым безработным студентам, предоставивщим справку о том, что они учаться в данном университете, выдают кредит.

<a id='conclusion'></a>
### Шаг 4. Общий вывод


На первых этапах был проведен анализ данных, обнаружены ошибки, аномальные данные, пропуски. 


Пропуски были обнаружены в столбцах total_income и days_employed. Как стало понятно из исследования таблицы и поисков дубликатов в дальнейшем, эти два столбца связаны, скорее всего формулой. Именно поэтому они дают пропуски в одинаковых строках и имеют дубликаты в некоторых строках. Я выдвинула гипотезу, о возникновении пропусков в данных столбцах. Люди, которые занимают руководящие должности, а так же имеют образование высшее образование чаще, чем другие не отвечают на вопрос о своих доходах. Мною было решено заполнить пропуски средним значением по столбцу, так как исключать эти данные из таблицы неприемлемо - они занимают 10% от всех данных, что даст искажение при исключении на дальнейшие исследования и выводы.


Обработала отрицательные значения, которые были в столбцах children и days_employed. Я предположила, что ошибка возникла в столбце children из-за лишнего символа "тире", который компьютер воспринял как символ "минус". А в столбце days_employed отрицательные значения появились из-за неверного пересчета, так как в трудовой книжке запись ведется в виде ДД.ММ.ГГГГ, а в нашей таблице данные храняться в днях.



Были обработаны аномальные значения в столбце days_employed. Я посчитала сколько в среднем человек может работать за всю свою жизнь, и взяла это число, как максимальное. Данные, которые были больше этого максимума были заменены на среднее значение. Такие аномальные числа появились из-за того, что данные могли браться из разных источников, и запись велась не в днях, а в часах или минутах.



Была проведена замена типов данных для удобства операций с данными. Так же в столбце с образованием и семейным статусом данные приведены в одному регистру, для дальнейших операций с ними.



Был создан новый столбец с конкретизированными целями, для дальнейшего исследования. Так же создан столбец для группировки заемщиков по заработной плате.


Из проведенного анализа данных получила зависимость между количеством детей и возвратом кредита в срок, что довольно логично, так как дети - всегда большие траты.

Отмечу, что одинокие люди, которые не состоят в отношениях, а так же люди, находящиеся в гражданском браке более склонны к невозврату кредита в срок.

Так же, заемщики, взявщие деньги на образование и автомобиль, чаще возвращают кредит не в срок, нежели чем на другие цели.